In [2]:
#计算仓库pr数量
def parse_pull_requests(file_path):
    repo_pr_map = {}

    with open(file_path, 'r') as file:
        previous_line = None

        for line in file:
            line = line.strip()
            if "pull requests" in line:
                # Store the previous line containing repo and PR count
                previous_line = line
            elif line.startswith("Done:") and previous_line:
                # Extract repo and PR count from the previous line
                repo_name, pr_info = previous_line.split(":")
                pr_count = int(pr_info.split()[0])  # Get the number of pull requests
                repo_pr_map[repo_name] = pr_count
                previous_line = None  # Reset the previous line

    # Sort the repositories by pull request count in descending order
    sorted_repos = sorted(repo_pr_map.items(), key=lambda x: x[1], reverse=True)

    # Print the sorted repositories
    for repo, pr_count in sorted_repos:
        print(f"{repo}: {pr_count} pull requests")

# Example usage
file_path = '/home/riv3r/SWE-bench/swebench/collect/logs/pr_log'
parse_pull_requests(file_path)


MaterializeInc/materialize: 21857 pull requests
MystenLabs/sui: 16593 pull requests
denoland/deno: 14393 pull requests
risingwavelabs/risingwave: 13209 pull requests
vectordotdev/vector: 12743 pull requests
tikv/tikv: 12368 pull requests
influxdata/influxdb: 11628 pull requests
pola-rs/polars: 10805 pull requests
zed-industries/zed: 10564 pull requests
rust-lang/rust-analyzer: 10522 pull requests
paritytech/substrate: 9903 pull requests
paradigmxyz/reth: 9766 pull requests
diem/diem: 9733 pull requests
bevyengine/bevy: 8966 pull requests
astral-sh/ruff: 8957 pull requests
rust-lang/crates.io: 8730 pull requests
near/nearcore: 8365 pull requests
apache/datafusion: 8127 pull requests
nushell/nushell: 7938 pull requests
bytecodealliance/wasmtime: 7185 pull requests
solana-labs/solana-program-library: 6755 pull requests
habitat-sh/habitat: 6735 pull requests
rust-lang/rust-clippy: 6707 pull requests
build-trust/ockam: 6686 pull requests
ruffle-rs/ruffle: 6662 pull requests
cube-js/cube: 63

In [ ]:
import requests
import os
GITHUB_API_URL = "https://api.github.com"

def get_pull_request(repo_owner, repo_name, pr_number, access_token=None):
    url = f"{GITHUB_API_URL}/repos/{repo_owner}/{repo_name}/pulls/{pr_number}"
    headers = {"Accept": "application/vnd.github.v3+json"}
    if access_token:
        headers["Authorization"] = f"Bearer {access_token}"
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f"PR fetched: {response.json()}")
        return response.json()  # 返回 PR 的详细信息
    else:
        print(f"Failed to fetch PR: {response.status_code} - {response.json()}")
        return None
    
a = get_pull_request("tokio-rs","tokio", "3965",os.getenv('GITHUB_TOKEN')) 


In [ ]:
def get_function_code(repo, commit, file_path, location, access_token=None):
    """
    获取指定仓库某个提交中某文件的函数代码。

    :param repo: str, 仓库名称 (格式: owner/repo_name)
    :param commit: str, 提交哈希值或分支名称
    :param file_path: str, 文件的路径
    :param location: tuple, 函数所在的起始行和结束行 (start_line, end_line)
    :param access_token: str, GitHub 的个人访问令牌（可选）
    :return: str, 函数的代码
    """
    # 构建 GitHub API 请求 URL
    url = f"{GITHUB_API_URL}/repos/{repo}/contents/{file_path}?ref={commit}"
    headers = {"Accept": "application/vnd.github.v3+json"}
    if access_token:
        headers["Authorization"] = f"Bearer {access_token}"

    # 获取文件内容
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch file: {response.status_code} - {response.json()}")
        return None

    # 解码文件内容 (Base64 编码)
    file_data = response.json()
    file_content = base64.b64decode(file_data["content"]).decode("utf-8")

    # 提取函数代码
    start_line, end_line = location
    file_lines = file_content.splitlines()
    function_code = "\n".join(file_lines[start_line - 1:end_line])  # 注意行号从 1 开始

    return function_code

# Example usage
import base64
repo = "tokio-rs/tokio"
commit = "aef2d64b0a519ff6726f8c139ee1d3e6b1959b0b"
file_path = "tokio/Cargo.toml"
location = (1, 70)
access_token = os.getenv('GITHUB_TOKEN')
code = get_function_code(repo, commit, file_path, location, access_token)
print(code)

In [21]:
import re
from datasets import load_dataset
dataset = load_dataset("r1v3r/RustGPT_Bench_verified",split="train")

def extract_file_paths_and_locations(patch_text):
    """
    解析 unidiff 格式的 patch, 提取旧文件的文件路径和被修改的行号。

    返回旧文件路径列表和对应文件的修改行号字典。
    """
    file_paths = []
    locations = {}
    current_file = None
    
    for line in patch_text.splitlines():
        if line.startswith('--- '):
            # 原文件路径
            original_file = line[6:].strip()
            current_file = original_file
            if current_file.endswith('.rs'):
                file_paths.append(current_file)
            locations[current_file] = []
        elif line.startswith('@@ '):
            # hunks，提取旧文件的行号范围
            match = re.match(r'@@ -(\d+)(?:,(\d+))? \+\d+(?:,\d+)? @@', line)
            if match:
                start_line = int(match.group(1))
                line_count = int(match.group(2)) if match.group(2) else 1
                location = (start_line, start_line + line_count)
                locations[current_file].append(location)
    return file_paths, locations



print(dataset[0]["instance_id"])
file_paths, locations = extract_file_paths_and_locations(dataset[0]["patch"])
print(file_paths)
print(locations)




tokio-rs__tokio-3965
['tokio/src/runtime/task/harness.rs']
{'tokio/src/runtime/task/harness.rs': [(420, 427)]}


In [22]:
locations = {
    'tokio/src/runtime/task/harness.rs': [(420, 427)(420, 427)(420, 427)(420, 427)(420, 427)(420, 427)],
    'tokio/src/runtime/task/harness.rs': [(420, 427)(420, 427)]
}


<>:2: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:2: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_4213/4209468273.py:2: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  'tokio/src/runtime/task/harness.rs': [(420, 427)(420, 427)(420, 427)(420, 427)(420, 427)(420, 427)],
/tmp/ipykernel_4213/4209468273.py:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  'tokio/src/runtime/task/harness.rs': [(420, 427)(420, 427)]


TypeError: 'tuple' object is not callable

In [24]:
import pysnooper

file_path = file_paths[0]
commit = dataset[0]["base_commit"]
repo = dataset[0]["repo"]

url = f"{GITHUB_API_URL}/repos/{repo}/contents/{file_path}?ref={commit}"
headers = {"Accept": "application/vnd.github.v3+json"}
if access_token:
    headers["Authorization"] = f"Bearer {access_token}"

# 获取文件内容
response = requests.get(url, headers=headers)
if response.status_code != 200:
    print(f"Failed to fetch file: {response.status_code} - {response.json()}")

# 解码文件内容 (Base64 编码)
file_data = response.json()
file_content = base64.b64decode(file_data["content"]).decode("utf-8")

print(file_content)

use crate::future::Future;
use crate::runtime::task::core::{Cell, Core, CoreStage, Header, Scheduler, Trailer};
use crate::runtime::task::state::Snapshot;
use crate::runtime::task::waker::waker_ref;
use crate::runtime::task::{JoinError, Notified, Schedule, Task};

use std::mem;
use std::panic;
use std::ptr::NonNull;
use std::task::{Context, Poll, Waker};

/// Typed raw task handle
pub(super) struct Harness<T: Future, S: 'static> {
    cell: NonNull<Cell<T, S>>,
}

impl<T, S> Harness<T, S>
where
    T: Future,
    S: 'static,
{
    pub(super) unsafe fn from_raw(ptr: NonNull<Header>) -> Harness<T, S> {
        Harness {
            cell: ptr.cast::<Cell<T, S>>(),
        }
    }

    fn header(&self) -> &Header {
        unsafe { &self.cell.as_ref().header }
    }

    fn trailer(&self) -> &Trailer {
        unsafe { &self.cell.as_ref().trailer }
    }

    fn core(&self) -> &Core<T, S> {
        unsafe { &self.cell.as_ref().core }
    }

    fn scheduler_view(&self) -> SchedulerView<'_,